import requirement

In [ ]:
import os, sys, glob, csv
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
import xlsxwriter
import random
from random import shuffle
from math import log, floor
import re
import collections
from collections import Counter
import string
import unicodedata as udata
import pause, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from distutils.dir_util import copy_tree
import sklearn
from sklearn.metrics import *
import itertools as it

# statistics

計算list中的各項統計資料

In [ ]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

計算總共有幾個family、Tree、samples、processes

In [ ]:
def statistic_hash_pids(root_dir):
    '''
    dir hierachical: root_dir/family_dirs/trees/XXX.profiles
    '''
    fam_dir = next(os.walk(root_dir))[1]
    all_pids= []
    all_pid_list = []
    tree_count = 0
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        tree_count += len(tree_dir)
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            hash_list = [f.split('/')[-1].split('_')[0] for f in hl_list]
            pid_list = [f.split('/')[-1] for f in hl_list]
            all_pids.extend(hash_list)
            all_pid_list.extend(pid_list)
#             print(fam,len(hash_list)) #DEBUG
    all_hash = set(all_pids)
    print('Families#:',len(fam_dir),'Samples#:',len(all_hash),',Processes#:',len(set(all_pid_list)),',Trees#:',tree_count)
    return all_hash, list(set(all_pid_list))

將list 或是 dict 輸出至excel當中畫分布圖

In [ ]:
def output_xlsx(statistics,i):
    '''
    Input: list or dict. i for iterations
    Output: xlsx
    '''
    output_root_path = 'data/tree-rep-profiles_o2o/df' + str(i+1) + '.xlsx'
    try:
        df = basic_statistics(statistics)
    except TypeError:
        df = basic_statistics(list(statistics.values()))
    df.to_excel(output_root_path,index=True,sheet_name='df'+str(i+1))
    return df

某個資料夾下的各項統計數值計算: process長度、rep長度、tree樣本數、tree執行程序數、一個sample有幾個process、一個family有幾個processes、一個family有幾個samples、一個family有幾個trees

(包含小樹短樹的整體統計or僅正常樹的統計)

In [ ]:

root_dir = './data/tree-rep-profiles_one2one/'
'''
root_dir hierachy: root_dir/normal(or small_short)/family_dirs/tree_dirs/XXX.profiles
'''
# api_length = []
process_lengths_normal = [] #濾掉小行為樹與短行為樹最後所剩之processes長度統計
process_lengths_overall = [] #包含小行為樹與短行為樹整體之processes長度統計
rep_length_normal = []  #濾掉小行為樹與短行為樹最後所剩行為樹之representative call sequence長度
rep_length_overall = [] #包含小行為樹與短行為樹之representative call sequence長度
tree_samples_normal = [] #濾掉小行為樹與短行為樹最後剩下行為樹所包含之sample數量統計
tree_samples_overall = [] #包含小行為樹與短行為樹之sample數量統計
tree_processes_normal = [] #濾掉小行為樹與短行為樹最後剩下行為樹所包含之sample數量統計
tree_processes_overall = [] #包含小行為樹與短行為樹之processes數量統計
sample_processes_normal = {} #濾掉小行為樹與短行為樹後一個sample所擁有的process數量
sample_processes_overall = {} #包含小行為樹與短行為樹一個sample所擁有的process數量
fam_processes_normal = [] #濾掉小行為樹與短行為樹後一個Family所含之processes數量
fam_processes_overall = dict.fromkeys(fam_dir_normal,0) #包含小行為樹與短行為樹一個Family所含之processes數量
fam_samples_normal = [] #濾掉小行為樹與短行為樹後一個Family所含之samples數量
fam_samples_overall = dict.fromkeys(fam_dir_normal,0) #包含小行為樹與短行為樹一個Family所含之samples數量
# processed_profile = []
trees_normal = [] #濾掉小行為樹與短行為樹最後所剩Family所含之行為樹個數統計
trees_overall = dict.fromkeys(fam_dir_normal,0)  #包含小樹短樹Family所含之行為樹個數統計

rasmma_dir = next(os.walk(root_dir))[1]
for rasmma in rasmma_dir:
    fam_dir = next(os.walk(root_dir+rasmma))[1]
    for fam in tqdm(fam_dir):
        if fam not in fam_dir_normal:
            continue
        tree_dir = next(os.walk(root_dir +rasmma+'/'+ fam))[1]
        if rasmma == 'normal':
#             assert len(tree_dir)>2
            trees_normal.append(len(tree_dir))
        trees_overall[fam] = trees_overall[fam] + len(tree_dir)
        fam_processes = 0
        fam_sample = []
        fam_sample_all = []
        for tree in tree_dir:
            in_directory = root_dir+ rasmma+'/'+ fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            rep = pickle.load(open(in_directory + 'rep.pickle','rb'))
            rep = sum(rep,[])
            fam_processes_overall[fam] = fam_processes_overall[fam] + len(hl_list)
            if rasmma == 'normal':
                assert len(rep) > 10
                assert len(hl_list) > 2
                rep_length_normal.append(len(rep))
                tree_processes_normal.append(len(hl_list))
                fam_processes = fam_processes + len(hl_list)
            rep_length_overall.append(len(rep))
            tree_processes_overall.append(len(hl_list))
            
            tree_samples = []
            for profile in hl_list:
                hash_sample = profile.split('/')[-1].split('_')[0]
                tree_samples.append(hash_sample)
                try:
                    sample_processes_overall[hash_sample] = sample_processes_overall[hash_sample] + 1
                except KeyError:
                    sample_processes_overall[hash_sample] = 1
                
#                 tree_samples_overall.append(hash_sample)
#                 name = profile.split('/')[-1]
#                 if name not in processed_profile:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
                lines = re.sub('y\\n','',lines)
                lines = re.sub('=\\n','',lines)
                lines = lines.splitlines()
#                     processed_profile.append(name)
                fam_sample_all.append(hash_sample)
                if rasmma == 'normal':
                    process_lengths_normal.append(len(lines))
                    fam_sample.append(hash_sample)
                    try:
                        sample_processes_normal[hash_sample] = sample_processes_normal[hash_sample] + 1
                    except KeyError:
                        sample_processes_normal[hash_sample] = 1
                process_lengths_overall.append(len(lines))
            tree_samples_overall.append(len(set(tree_samples)))
            if rasmma == 'normal':
                tree_samples_normal.append(len(set(tree_samples)))
        fam_samples_overall[fam] = fam_samples_overall[fam] + len(set(fam_sample_all))
        if rasmma == 'normal':
            fam_processes_normal.append(fam_processes)
            fam_samples_normal.append(len(set(fam_sample)))
#                 else:
#                     paths = glob.glob(root_dir + '*/*/*/'+profile.split('/')[-1])
#                     print('duplicate ERR:',paths)

assert len(trees_normal) == len(fam_dir_normal) == len(fam_processes_normal) == len(fam_samples_normal)
assert sum(trees_normal) == len(tree_samples_normal) == len(tree_processes_normal)
# len(processed_profile)

In [ ]:
df_all_list = []
for i,stat in enumerate([trees_normal,trees_overall,process_lengths_normal,process_lengths_overall
                         ,rep_length_normal,rep_length_overall,tree_samples_normal,tree_processes_normal,
                        tree_samples_overall,tree_processes_overall,fam_samples_normal,fam_processes_normal,
                        fam_samples_overall,fam_processes_overall,sample_processes_normal,sample_processes_overall]):
    df_all_list.append(output_xlsx(stat,i))

# Preprocess

## outlier removal
* 把profile從root_dir移動到mv_dir

In [ ]:
outlier_len_profile = df_all_list[2].loc['outlier'].values[0] #outlier
print(np.percentile(process_lengths_normal,82)) #總共涵蓋?比例的profiles ?要自己調到最接近outlier value
outlier_len_profile 

In [ ]:
mv_dir = './data/tree-rep-profiles_o2o/5.long/' #要先自己創立移動目的地
root_dir = './data/tree-rep-profiles_o2o/normal/' #原始要被篩選的資料夾
fam_dir = next(os.walk(root_dir))[1]
remove_pids = 0
nobyteseq = 0

for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for profile in hl_list:
            with open(profile,encoding='ISO 8859-1') as f:
                lines = f.read().splitlines()
            if len(lines)> outlier_len_profile:
                dest_path = mv_dir + fam + '/' + tree + '/'
                if not os.path.exists(dest_path):
                    os.makedirs(dest_path,exist_ok=True)
                shutil.move(profile,dest_path+profile.split('/')[-1])
                byteseq = profile.split('.profile')[0] + '_byterep.pickle'
                try:
                    shutil.move(byteseq,dest_path+byteseq.split('/')[-1])
                except FileNotFoundError:
                    print(fam,tree)
                    nobyteseq +=1
                remove_pids +=1
print('remove processes:',remove_pids,'No byteseq:',nobyteseq)

In [ ]:
#此block要多執行幾次直到把空fam或是tree皆移除 (i.e., Empty Tree#: 0 Empty Fam#: 0)
root_dir = './data/tree-rep-profiles_o2o/normal/'
fam_dir = next(os.walk(root_dir))[1]
empty_tree = 0
empty_fam = 0
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    fam_tree = {}
    if len(tree_dir) == 0:
        shutil.move(root_dir + fam  ,mv_dir+fam)
        empty_fam +=1
        print(fam)
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        if len(hl_list) == 0: #移除tree底下的member僅有0個者
            shutil.move(root_dir + fam +  '/' + tree ,mv_dir+fam+'/'+tree)
            empty_tree +=1
print('Empty Tree#:',empty_tree,'Empty Fam#:',empty_fam)


all_hash,all_pid_list = statistic_hash_pids(root_dir) # 65.smshoax_0.8 148.dldpk_0.8 114.rbot_0.8 147.ranpax_0.8 107.jorik_0.8

## Train, Dev, Test split
* 拿tree數量sample的10%給valid、10%給test
* profile\*10%不足1的tree跳過

In [ ]:
root_dir = './data/tree-rep-profiles_o2o/normal/' #原始p前處理後的rofile資料夾
fam_dir = next(os.walk(root_dir))[1]

valid_dir = './data/tree-rep-profiles_o2o/DEV/' #validation 資料夾要先創好
test_dir = './data/tree-rep-profiles_o2o/TEST/' #testing 資料夾要先創好

In [ ]:
# 此做法為還原tree，依照process來分

for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
#         hash_list = list(set([x.split('/')[-1].split('_')[0] for x in hl_list]))
        if len(hl_list)>10:# 從各tree拿一個sample給dev一個給test
#             dest_path = valid_dir + fam + '/' + tree + '/'
#             if not os.path.exists(dest_path):
#                 os.makedirs(dest_path,exist_ok=True)
            inf_idx = random.sample(range(0,len(hl_list)),int(len(hl_list)*0.2))
            valid_idx = random.sample(inf_idx,int(len(inf_idx)/2))
            test_idx = list(set(inf_idx)-set(valid_idx))
            valid_paths = [hl_list[x] for x in valid_idx]
            test_paths = [hl_list[x] for x in test_idx]
#             valid_paths = []
#             for valid in valid_pid:
#                 paths = glob.glob(root_dir + '*/*/' + valid + '*')
#                 valid_paths.extend(paths)
#             test_paths = []
#             for test in test_pid:
#                 paths = glob.glob(root_dir + '*/*/' + test + '*')
#                 test_paths.extend(paths)  
            for source_path in valid_paths:
                dest_path = source_path.replace(root_dir.split('/')[-2],valid_dir.split('/')[-2])
                dev_dir = '/'.join(dest_path.split('/')[:-1]) + '/'
                if not os.path.exists(dev_dir):
                    os.makedirs(dev_dir,exist_ok=True)
                shutil.move(source_path,dest_path)
                source_path = source_path.replace('.profile','_byterep.pickle')
                dest_path = dest_path.replace('.profile','_byterep.pickle')
                shutil.move(source_path,dest_path)
            for source_path in test_paths:
                dest_path = source_path.replace(root_dir.split('/')[-2],test_dir.split('/')[-2])
                dev_dir = '/'.join(dest_path.split('/')[:-1]) + '/'
                if not os.path.exists(dev_dir):
                    os.makedirs(dev_dir,exist_ok=True)
                shutil.move(source_path,dest_path)
                source_path = source_path.replace('.profile','_byterep.pickle')
                dest_path = dest_path.replace('.profile','_byterep.pickle')
                shutil.move(source_path,dest_path)            

train/dev/test所含的family/tree/sample/process數量

In [ ]:
temp = statistic_hash_pids(root_dir)
temp = statistic_hash_pids(valid_dir)
temp = statistic_hash_pids(test_dir)

每個dataset所含的各家族之tree/sample/process數量

In [ ]:
# initialize
train_dir = './data/tree-rep-profiles_o2o/normal/' #Train Set
valid_dir = './data/tree-rep-profiles_o2o/DEV/' # DEV Set
test_dir = './data/tree-rep-profiles_o2o/TEST/' # Test Set
fam_dir = next(os.walk(train_dir))[1]
rows = [x.split('_')[0].split('.')[1] for x in fam_dir]
columns = ['train_trees_num','train_samples_num','train_processes_num',
          'valid_trees_num','valid_samples_num','valid_processes_num',
          'test_trees_num','test_samples_num','test_processes_num']
stat_df = pd.DataFrame(0, index=rows,columns=columns)

In [ ]:
# 需要iterate多次計算才可以得到三個dataset的總表
def stat_fam(root_dir,root_trees_num,root_samples_num,root_processes_num,stat_df=stat_df):
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        trees_num = len(tree_dir)
        fam_name = fam.split('_')[0].split('.')[1]
        profiles_num = 0
        samples_num_li = []
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            profiles_num = profiles_num + len(hl_list)
            hash_list = list(set([x.split('/')[-1].split('_')[0] for x in hl_list]))
            samples_num_li.extend(hash_list)
#             samples_num = samples_num + len(hash_list)
        stat_df.loc[fam_name,root_trees_num] = trees_num
        stat_df.loc[fam_name,root_samples_num] = len(set(samples_num_li))
        stat_df.loc[fam_name,root_processes_num] = profiles_num
    return stat_df

In [ ]:
stat_df = stat_fam(train_dir,'train_trees_num','train_samples_num','train_processes_num',stat_df=stat_df)
stat_df = stat_fam(valid_dir,'valid_trees_num','valid_samples_num','valid_processes_num',stat_df=stat_df)
stat_df = stat_fam(test_dir,'test_trees_num','test_samples_num','test_processes_num',stat_df=stat_df)
stat_df.to_excel('./data/tree-rep-profiles_o2o/train_valid_test_fam_stat_correct.xlsx') # 儲存至excel位置 (資料夾要先創好)
stat_df

# 準備實驗3要用的Loner Set (thesis sec. 4.4)

 選取loner有在dev跟test裡面的fam就好

In [ ]:
root_dir = './data/tree-rep-profiles_o2o/DEV/' #要選取的依據家族
loner_dir = './data/tree-rep-profiles_o2o_loner/' #原始loner Set資料夾 (這個資料很髒，有很多的empty file或是過短的file可能要自行額外處理)
exp_dir = './data/tree-rep-profiles_o2o/EXP2/' #要存放的目的資料夾
# outlier_len_profile = 226

exp_fam = next(os.walk(root_dir))[1] # 要進行家族歸類的家族
exp_fam_dir = [x.split('_')[0] for x in exp_fam]
# aliase_fam = [x.split('.')[-1] for x in exp_fam_dir]
len(exp_fam_dir),exp_fam_dir #, aliase_fam

In [ ]:
outlier_profile = []
for fam in tqdm(exp_fam_dir):
    in_directory =  loner_dir + fam + '/G0/'
    profiles = next(os.walk(in_directory))[2]  # 可能某些familiy會沒loner
    hl_list = [os.path.join(in_directory, f) for f in profiles]
    hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
    for profile in hl_list:
        with open(profile,encoding='ISO 8859-1') as f:
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines() #Ri
        if len(lines) > outlier_len_profile: #太長的放不進去model 需要紀錄?
            outlier_profile.append(profile)
            continue
        profile_name = profile.split('/')[-1]
        fam_name = profile.split('/')[3].split('.')[-1]

        dest_dir = exp_dir + fam_name + "/G0/"
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir,exist_ok=True)
        shutil.copy(profile,dest_dir+profile_name)
# 所移動到的exp_dir仍很髒，需要再進一步處理才可以拿去做使用，當前的統計資訊不會是對的或是最終採用的
# 建議自己這邊先額外寫前處理，篩選出要的profile就好再往0003前進

In [ ]:
# 移除了多少個太長的loner?
outlier_profile_hash = [x.split('/')[-1].split('_')[0] for x in outlier_profile]
outlier_profile_fam = [x.split('/')[3] for x in outlier_profile]
print(len(set(outlier_profile_hash)),len(outlier_profile),len(set(outlier_profile_fam))) #sample#,process#,哪一些家族有發生

## 選擇Sent2Vec演算法要用的n-gram windows size 要多大

In [ ]:
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" 
root_dir = './data/tree-rep-profiles_o2o/normal/' #訓練資料集資料夾/families/trees/profiles
fam_dir = next(os.walk(root_dir))[1]
api_length = []
profile_length = []

for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for profile in hl_list:
            with open(profile,encoding='ISO 8859-1') as f: #X2
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
            lines = re.sub('=\\n','',lines)
            lines = re.sub('y\\n','',lines)
            lines = lines.splitlines()
            for line in lines:
                temp = re.sub(dil," ",line.lower())
                temp = temp.split(" ")
                temp = list(filter(None, temp))
                api_length.append(len(temp))

In [ ]:
basic_statistics(api_length) #選取眾數(mode)做為Sent2Vec演算法的超參數 n 

***

# Appendix Spplementary
* 與任務主線無關，只是實驗用的小工具
* Deprecated code. Need lots of modifications

主線是依據tree的process來分，以下是以sample來分

In [ ]:
# 此做法為希望系統可以還原family，而非評估individual tree rep，依照sample來分並且會拿process
'''for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        hash_list = list(set([x.split('/')[-1].split('_')[0] for x in hl_list]))
        if len(hash_list)>10:# 從各tree拿一個sample給dev一個給test
#             dest_path = valid_dir + fam + '/' + tree + '/'
#             if not os.path.exists(dest_path):
#                 os.makedirs(dest_path,exist_ok=True)
            inf_idx = random.sample(range(0,len(hash_list)),int(len(hash_list)*0.2))
            valid_idx = random.sample(inf_idx,int(len(inf_idx)/2))
            test_idx = list(set(inf_idx)-set(valid_idx))
            valid_hash = [hash_list[x] for x in valid_idx]
            test_hash = [hash_list[x] for x in test_idx]
            valid_paths = []
            for valid in valid_hash:
                paths = glob.glob(root_dir + '*/*/' + valid + '*')
                valid_paths.extend(paths)
            test_paths = []
            for test in test_hash:
                paths = glob.glob(root_dir + '*/*/' + test + '*')
                test_paths.extend(paths)  
            for source_path in valid_paths:
                dest_path = source_path.replace(root_dir.split('/')[-2],valid_dir.split('/')[-2])
                dev_dir = '/'.join(dest_path.split('/')[:-1]) + '/'
                if not os.path.exists(dev_dir):
                    os.makedirs(dev_dir,exist_ok=True)
                shutil.move(source_path,dest_path)
            for source_path in test_paths:
                dest_path = source_path.replace(root_dir.split('/')[-2],test_dir.split('/')[-2])
                dev_dir = '/'.join(dest_path.split('/')[:-1]) + '/'
                if not os.path.exists(dev_dir):
                    os.makedirs(dev_dir,exist_ok=True)
                shutil.move(source_path,dest_path)
            '''

loner Set 家族名稱別名分析與合併

In [ ]:
df = pd.read_csv('./data/FamilyName_aliases.csv') # 要先自己創好，範例放在google drive的table裡面FamilyName_aliases
df = df.set_index('Unnamed: 0')
name_df_aliases = df[aliase_fam]
name_df_aliases

In [ ]:
#aliases
print('sklearn version need >0.21 now:',sklearn.__version__)
aliases_li = []
all_process_fam = []
for a,b in it.combinations(name_df_aliases.columns.tolist(),2): 
    score = jaccard_score(name_df_aliases.loc[:,a].values , name_df_aliases.loc[:,b].values)
    if score > 0.6: #可改相似度thr
        print(a,b,score)
        if (a in all_process_fam) and (b not in all_process_fam):
            for aliases in aliases_li:
                if a in aliases:
                    aliases_li.remove(aliases)
                    aliases.append(b)
                    aliases_li.append(aliases)
                    all_process_fam.append(b)
        elif (b in all_process_fam) and (b not in all_process_fam):
            for aliases in aliases_li:
                if b in aliases:
                    aliases_li.remove(aliases)
                    aliases.append(a)
                    aliases_li.append(aliases)
                    all_process_fam.append(a)
        elif (a in all_process_fam) and (b in all_process_fam):
            pass
        else:
            aliases_li.append([a,b])
            all_process_fam.append(b)
            all_process_fam.append(a)
# ['allaple','rahack']
# []
aliases_li

In [ ]:
outlier_profile = []
for fam in tqdm(exp_fam_dir):
    in_directory =  loner_dir + fam + '/G0/'
    profiles = next(os.walk(in_directory))[2]  # 可能某些familiy會沒loner
    hl_list = [os.path.join(in_directory, f) for f in profiles]
    hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
    for profile in hl_list:
        with open(profile,encoding='ISO 8859-1') as f:
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines() #Ri
        if len(lines) > outlier_len_profile: #太長的放不進去model 需要紀錄?
            outlier_profile.append(profile)
            continue
        profile_name = profile.split('/')[-1]
        fam_name = profile.split('/')[3].split('.')[-1]
        for aliases in aliases_li:
            if fam_name in aliases:
                fam_name = aliases[0]+'_'+aliases[1] #aliases analyze and combine

        dest_dir = exp_dir + fam_name + "/G0/"
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir,exist_ok=True)
        shutil.copy(profile,dest_dir+profile_name)
temp = statistic_hash_pids(exp_dir) #真正拿來做實驗的loners

### Loners similarity computation
* 取出實驗家族exp_fam所含有的profiles
* 利用RasMMA計算是否是跟自己家族的行為樹最相近
* 如果是就把它移到exp_dir
* Deprecation原因(需要修改以下程式碼來達到下兩點目的):
    * 不應該profile跟rep比
    * 應該要profile跟rep(tree)的memeber比

In [ ]:
root_dir = './data/tree-rep-profiles_o2o/DEV/'
loner_dir = './data/tree-rep-profiles_o2o_loner/'
exp_dir = './data/tree-rep-profiles_o2o/EXP_rasmma/'

exp_fam = next(os.walk(root_dir))[1] #改align root_dir? 要進行家族歸類的家族
exp_fam_dir = [x.split('_')[0] for x in exp_fam]
len(exp_fam_dir),exp_fam_dir

RasMMA similarity function
* 需要有偉志撰寫經我修改後的相關程式碼檔案(放在github)

In [ ]:
%run Alignment_Fast3.ipynb

# Doing global alignment and find commonAPISequence.
def do_globalAlignment(rep1, rep2):
    '''
    Input: 兩個1D list (profile(rep1) , rep(rep2))
    Output: 一個1D list (common)
    '''
    # Aligment
    commonAPISequence = []
    alignment_result = globalAlign( rep1, rep2, score_matched=10, score_mismatched=-1, score_gap=0)[2]
    common_motif_sequence = motif_delimit(alignment_result)
    return common_motif_sequence

# input: two R
# output: new Rep's common motif sequence of input CMS;
def get_Rep_CommMotifSeq(Ri, Rj):
    '''
    Input: 兩個1D list (profile(Ri) , rep(Rj))
    Output: common motif 1D list
    '''
    rep1 = Ri
    rep2 = Rj
    repNew_CMS = []
#     print(rep2)
    if(rep1 and rep2):
#         print("===in rep1 and rep2===")
        commonSequence = do_globalAlignment(rep1, rep2) # do Alignment
#         print(commonSequence)
        repNew = commonSequence
    else: ##ADD
        repNew = [] #ADD
#     print('===out===')
    return repNew

# compute score of Rnew
# the score calculate method is the length ratio of new to origin one
def compute_Score(Ri, Rj, Rnew):
    '''
    Input: 3個 1D list (Profile(Ri), Rep(Rj), Common(Rnew))
    Output: 相似度分數
    '''
    if(Rnew):
        repI = Ri
        repJ = Rj
        repNew = Rnew
        L_Ri = len(repI)
        L_Rj = len(repJ)
        Lorg = max(L_Ri, L_Rj)
        Lnew = len(repNew)
        return float(Lnew)/Lorg
    else:
        return 0

In [ ]:
count_accept_loner = 0 # 1 hr
accept_loner_paths = []
max_scores = []

for fam in tqdm(exp_fam_dir):
    in_directory = loner_dir+ fam+'/' + 'G0' + '/'
    profiles = next(os.walk(in_directory))[2]  # 可能某些familiy會沒loner
    hl_list = [os.path.join(in_directory, f) for f in profiles]
    hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
    for profile in hl_list:
        with open(profile,encoding='ISO 8859-1') as f:
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines() #Ri
        if len(lines) > outlier_len_profile: #太長的放不進去model
            continue
        ori_fam = fam+'_0.8'
        ori_rep_trees = next(os.walk(root_dir + ori_fam + '/'))[1] # 原所屬家族tree的rep找出來
        ori_scores = []
        for tree in ori_rep_trees:
            rep_directory = root_dir + ori_fam + '/' + tree + '/' #+ 'rep.pickle'
#             rep = pickle.load(open(rep_directory,'rb'))
#             rep = sum(rep,[])
            root_dir_tree_profiles = next(os.walk(rep_directory))[2]
            hl_list_ori = [os.path.join(root_dir_tree_profiles, f) for f in profiles]
            hl_list_ori = list(filter(lambda f: f.endswith(".profile"), hl_list_ori))
            for profile_ori in hl_list_ori:
                with open(profile_ori,encoding='ISO 8859-1') as f:
                    lines_ori = f.read()
                lines_ori = re.sub(r'[^\x00-\x7F]+','', lines_ori)
                lines_ori = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines_ori)
                lines_ori = re.sub('=\\n','',lines_ori)
                lines_ori = re.sub('y\\n','',lines_ori)
                lines_ori = lines_ori.splitlines()                
                common = get_Rep_CommMotifSeq(lines,lines_ori)
                score = compute_Score(lines,lines_ori,common)
            ori_scores.append(score)
        ori_max_score = max(ori_scores)
        flag = True
        rest_dir = list(set(exp_fam) - set([ori_fam]))
        for rep_dir in rest_dir:
            rep_trees = next(os.walk(root_dir + rep_dir + '/'))[1]
            for tree in rep_trees:
                rep_directory = root_dir + rep_dir + '/' + tree + '/' + 'rep.pickle'
                rep = pickle.load(open(rep_directory,'rb'))
                rep = sum(rep,[])
                common = get_Rep_CommMotifSeq(lines,rep)
                score = compute_Score(lines,rep,common)
                if score > ori_max_score: #loner跟別的家族tree分數比較高
                    flag = False
                    break
            if flag == False:
                break
        if (flag == True) :#and (score>0):
            accept_loner_paths.append(profile)
            max_scores.append(ori_max_score)
            count_accept_loner+=1
assert len(accept_loner_paths) == count_accept_loner == len(max_scores)
print(count_accept_loner)

In [ ]:
final_lon = []
final_hash_lon = []
final_fam_lon = []
final_score = []
for lon,score in zip(accept_loner_paths,max_scores):
    hash_lon = lon.split('/')[-1].split('_')[0]
    fam_name = lon.split('/')[3]
    if score>0:
        final_lon.append(lon)
        final_hash_lon.append(hash_lon)
        final_fam_lon.append(fam_name)
        final_score.append(score)
final_hash_lon = list(set(final_hash_lon))
assert len(final_hash_lon) <= len(final_lon)
print('Processes#:',len(final_lon),'Hash#:',len(final_hash_lon))
final_fam_lon = dict(Counter(final_fam_lon)) #loner
len(final_fam_lon) #loner

In [ ]:
min(final_score) , max(final_score) ,np.mean(final_score) , np.std(final_score) , np.percentile(final_score,50)

把最後loner跟原本自己家族夠相近的profile copy to root_dir

In [ ]:
for lon in tqdm(final_lon):
    dest_dir = exp_dir + lon.split('/')[3] +'/'+ 'G0/'
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir,exist_ok=True)
    shutil.copy(lon,dest_dir+lon.split('/')[-1])